In [1]:
# Import Dependencies
import requests
import json
import pandas as pd
import time
import numpy as np
from config import yelp_key

In [2]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [3]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [4]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','bedbreakfast','hotels','resorts',
              'skiresorts','tours', 'transportation','restaurants','nightlife'
             ]

len(categories)

21

In [5]:
# Import Lat/Long Database
csv_path = "../Resources/CitiesWGeolocation.csv"
locations_df = pd.read_csv(csv_path)
locations_df.head()

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.046407,-84.497039
1,1,San Diego,California,United States,32.717420,-117.162773
2,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871
3,3,Park City,Utah,United States,40.646092,-111.497996
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157


In [6]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.loc[i].at["Latitude"])
    longitudes_list.append(locations_df.loc[i].at["Longitude"])
print(latitudes_list)
print(longitudes_list)

[38.0464066, 32.7174202, -19.99697155, 40.646092100000004, 54.97384739999999, -27.468968199999996, -28.0402165, 34.244058700000004, -36.718804999999996, 49.5041747, 42.64873625, 37.2395367, -32.728465, 17.223472100000002, 25.265347100000003, 50.67108245, 21.721746, 36.508976000000004, 32.0564572, -16.484598300000002, -33.928992, -19.1421421, -33.934444, -28.648333299999997, -45.0321923, 33.77217945, -6.166490799999999, -20.2759451, 45.437190799999996, 19.70318225, -4.6574977, -28.002373100000003, 44.4654236, 31.6258257, 37.9374939, 49.8879177, 43.10656029999999, 30.2711224, 15.2214956, 43.4832523, -33.953177600000004, 26.9154576, 13.1500331, 20.169626800000003, 51.53882410000001, 60.39430550000001, 24.578720999999998, -34.61341495, 34.862942600000004, 18.1850507, 12.51756625, -35.14181285, -34.4175, -33.8611665, -16.9206657, 44.958452799999996, -26.6544338, -34.427808299999995, 51.08668970000001, 44.279621, -37.814217600000006, -31.955896399999997, 51.453802200000005, 44.841225, 27.175

In [7]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
for i in range(len(latitudes_list)):
        for x in range(len(categories)):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&radius=24140&categories=" + category + "&sort_by=best_match"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

4620


['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=beaches&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hanggliding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=horsebackriding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hiking&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hot_air_balloons&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=paddleboarding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=parasailing&sort_by=best_matc

In [8]:
# Create API Calls (separate the blank list to a new cell so we can do numerous calls if needed)
data_list=[]

In [9]:
record_count = 1
set_count = 1
url_errors = []

for i in range(len(url_list)):
    if (i%100 == 0 and i >=100):
        set_count += 1
        record_count = 0
        
    print("processing Record %s of Set %s | %s" %(record_count, set_count,url_list[i]))    
    
    record_count += 1
    try:
        response = requests.get(url_list[i],headers=headers)
        data = response.json()

        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        url_errors.append(url_list[i])
        pass

processing Record 1 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=beaches&sort_by=best_match
processing Record 2 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 3 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=horsebackriding&sort_by=best_match
processing Record 4 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hiking&sort_by=best_match
processing Record 5 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 6 of Set 1 | https://api.yelp.com/v3/businesses/search

********URL error: Skipping...********
processing Record 46 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 47 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 48 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=24140&categories=paddleboarding&sort_by=best_match
********URL error: Skipping...********
processing Record 49 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 50 of Set 1 | https://api.yelp.com/v3/businesses/search?lati

********URL error: Skipping...********
processing Record 87 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=horsebackriding&sort_by=best_match
processing Record 88 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=hiking&sort_by=best_match
processing Record 89 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 90 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=paddleboarding&sort_by=best_match
********URL error: Skipping...********
processing Record 91 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=parasailing&sort_by=be

processing Record 29 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-28.0402165&longitude=152.81060540815727&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 30 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-28.0402165&longitude=152.81060540815727&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 31 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-28.0402165&longitude=152.81060540815727&radius=24140&categories=paddleboarding&sort_by=best_match
processing Record 32 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-28.0402165&longitude=152.81060540815727&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 33 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-28.0402165&longitude=152.81060540815727&radius=24140&categories=sailing&sort_by=

processing Record 71 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-36.718804999999996&longitude=142.22095086226386&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 72 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-36.718804999999996&longitude=142.22095086226386&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 73 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-36.718804999999996&longitude=142.22095086226386&radius=24140&categories=paddleboarding&sort_by=best_match
********URL error: Skipping...********
processing Record 74 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-36.718804999999996&longitude=142.22095086226386&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 75 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-36.718804

...

processing Record 46 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=resorts&sort_by=best_match
processing Record 47 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=skiresorts&sort_by=best_match
processing Record 48 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=tours&sort_by=best_match
processing Record 49 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=transportation&sort_by=best_match
processing Record 50 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=restaurants&sort_by=best_match
processing Record 51 of Set 45 | https://api.yelp.com/v3/businesses/searc

********URL error: Skipping...********
processing Record 88 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=resorts&sort_by=best_match
********URL error: Skipping...********
processing Record 89 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 90 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=tours&sort_by=best_match
********URL error: Skipping...********
processing Record 91 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=transportation&sort_by=best_match
********URL error: Skipping...********
processing Record 92 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&

********URL error: Skipping...********
processing Record 29 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=hotels&sort_by=best_match
********URL error: Skipping...********
processing Record 30 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=resorts&sort_by=best_match
********URL error: Skipping...********
processing Record 31 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 32 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=tours&sort_by=best_match
********URL error: Skipping...********
processing Record 33 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=

processing Record 73 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=skiresorts&sort_by=best_match
processing Record 74 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=tours&sort_by=best_match
processing Record 75 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=transportation&sort_by=best_match
processing Record 76 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=restaurants&sort_by=best_match
processing Record 77 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=nightlife&sort_by=best_match
processing Record 78 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.

********URL error: Skipping...********
processing Record 13 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=24140&categories=hotels&sort_by=best_match
********URL error: Skipping...********
processing Record 14 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=24140&categories=resorts&sort_by=best_match
********URL error: Skipping...********
processing Record 15 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 16 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=24140&categories=tours&sort_by=best_match
********URL error: Skipping...********
processing Record 17 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=24140&categori

In [10]:
# convert the list of dictionaries to a dataframe
yelp_df = pd.DataFrame(data_list)   
yelp_df

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,4.500,2.00
1,38.046407,-84.497039,horsebackriding,8,1.125,0.25
2,38.046407,-84.497039,hiking,4,4.625,14.00
3,38.046407,-84.497039,paddleboarding,2,4.750,15.00
4,38.046407,-84.497039,ziplining,20,4.250,258.20
...,...,...,...,...,...,...
2192,46.201756,6.146601,restaurants,20,4.350,50.95
2193,46.201756,6.146601,nightlife,20,4.225,11.25
2194,35.898982,14.513676,ziplining,1,4.500,3.00
2195,35.898982,14.513676,transportation,1,4.500,3.00


In [11]:
url_errors

['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hanggliding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hot_air_balloons&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=parasailing&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=sailing&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=snorkeling&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hotsprings&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=skiresorts&sort_by=best_match',
 

In [ ]:
record_count = 1
set_count = 1
url_errors_p2 = []

for i in range(len(url_errors)):
    if (i%100 == 0 and i >=100):
        set_count += 1
        record_count = 0
        
    print("processing Record %s of Set %s | %s" %(record_count, set_count,url_errors[i]))    
    
    record_count += 1
    try:
        response = requests.get(url_errors[i],headers=headers)
        data = response.json()

        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        url_errors_p2.append(url_errors[i])
        pass

processing Record 1 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 2 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 3 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 4 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=sailing&sort_by=best_match
********URL error: Skipping...********
processing Record 5 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=snorkeling&sort_by=best_

********URL error: Skipping...********
processing Record 40 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 41 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=sailing&sort_by=best_match
********URL error: Skipping...********
processing Record 42 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=snorkeling&sort_by=best_match
********URL error: Skipping...********
processing Record 43 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=54.97384739999999&longitude=-1.6131572&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 44 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-27.468968199999996&

********URL error: Skipping...********
processing Record 78 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=49.5041747&longitude=-115.062867&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 79 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=49.5041747&longitude=-115.062867&radius=24140&categories=paddleboarding&sort_by=best_match
********URL error: Skipping...********
processing Record 80 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=49.5041747&longitude=-115.062867&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 81 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=49.5041747&longitude=-115.062867&radius=24140&categories=sailing&sort_by=best_match
********URL error: Skipping...********
processing Record 82 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=49.5041747&longitude=-115.062867&r

********URL error: Skipping...********
processing Record 16 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=17.223472100000002&longitude=-61.9554608&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 17 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=17.223472100000002&longitude=-61.9554608&radius=24140&categories=horsebackriding&sort_by=best_match
********URL error: Skipping...********
processing Record 18 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=17.223472100000002&longitude=-61.9554608&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 19 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=17.223472100000002&longitude=-61.9554608&radius=24140&categories=hot_air_balloons&sort_by=best_match
********URL error: Skipping...********
processing Record 20 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=17

********URL error: Skipping...********
processing Record 55 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=50.67108245&longitude=-1.3328042802764226&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 56 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=50.67108245&longitude=-1.3328042802764226&radius=24140&categories=snorkeling&sort_by=best_match
********URL error: Skipping...********
processing Record 57 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=50.67108245&longitude=-1.3328042802764226&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 58 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=50.67108245&longitude=-1.3328042802764226&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 59 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=21.72

...

********URL error: Skipping...********
processing Record 63 of Set 7 | https://api.yelp.com/v3/businesses/search?latitude=27.1752554&longitude=78.0098161&radius=24140&categories=bedbreakfast&sort_by=best_match
********URL error: Skipping...********
processing Record 64 of Set 7 | https://api.yelp.com/v3/businesses/search?latitude=27.1752554&longitude=78.0098161&radius=24140&categories=hotels&sort_by=best_match
********URL error: Skipping...********
processing Record 65 of Set 7 | https://api.yelp.com/v3/businesses/search?latitude=27.1752554&longitude=78.0098161&radius=24140&categories=resorts&sort_by=best_match
********URL error: Skipping...********
processing Record 66 of Set 7 | https://api.yelp.com/v3/businesses/search?latitude=27.1752554&longitude=78.0098161&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 67 of Set 7 | https://api.yelp.com/v3/businesses/search?latitude=27.1752554&longitude=78.0098161&radius=24140&catego

********URL error: Skipping...********
processing Record 2 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=32.7762719&longitude=-96.7968559&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 3 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=32.7762719&longitude=-96.7968559&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 4 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=27.763530199999998&longitude=-97.4033191&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 5 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=27.763530199999998&longitude=-97.4033191&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 6 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=27.763530199999998&longitude=-97.4

********URL error: Skipping...********
processing Record 41 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=35.600949799999995&longitude=-82.5540161&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 42 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=-18.1239696&longitude=179.0122737&radius=24140&categories=beaches&sort_by=best_match
********URL error: Skipping...********
processing Record 43 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=-18.1239696&longitude=179.0122737&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 44 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=-18.1239696&longitude=179.0122737&radius=24140&categories=horsebackriding&sort_by=best_match
********URL error: Skipping...********
processing Record 45 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=-18.1239696&longitude=179.01

********URL error: Skipping...********
processing Record 80 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=59.938731999999995&longitude=30.316228999999996&radius=24140&categories=massage&sort_by=best_match
********URL error: Skipping...********
processing Record 81 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=59.938731999999995&longitude=30.316228999999996&radius=24140&categories=bedbreakfast&sort_by=best_match
********URL error: Skipping...********
processing Record 82 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=59.938731999999995&longitude=30.316228999999996&radius=24140&categories=hotels&sort_by=best_match
********URL error: Skipping...********
processing Record 83 of Set 8 | https://api.yelp.com/v3/businesses/search?latitude=59.938731999999995&longitude=30.316228999999996&radius=24140&categories=resorts&sort_by=best_match
********URL error: Skipping...********
processing Record 84 of Set 8 | https://api.yelp.com/v3/businesses/search

********URL error: Skipping...********
processing Record 19 of Set 9 | https://api.yelp.com/v3/businesses/search?latitude=24.7736546&longitude=-78.0000547&radius=24140&categories=ziplining&sort_by=best_match
********URL error: Skipping...********
processing Record 20 of Set 9 | https://api.yelp.com/v3/businesses/search?latitude=24.7736546&longitude=-78.0000547&radius=24140&categories=spas&sort_by=best_match
********URL error: Skipping...********
processing Record 21 of Set 9 | https://api.yelp.com/v3/businesses/search?latitude=24.7736546&longitude=-78.0000547&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 22 of Set 9 | https://api.yelp.com/v3/businesses/search?latitude=24.7736546&longitude=-78.0000547&radius=24140&categories=massage&sort_by=best_match
********URL error: Skipping...********
processing Record 23 of Set 9 | https://api.yelp.com/v3/businesses/search?latitude=24.7736546&longitude=-78.0000547&radius=24140&catego

In [12]:
# determine if there are any missing reviews. At this point, there are missing reviews for four categories
yelp_df["category"].value_counts()

ziplining           158
transportation      158
restaurants         156
hotels              146
nightlife           146
tours               143
spas                141
massage             141
bedbreakfast        137
horsebackriding     134
hiking              128
beaches             118
resorts             111
sailing              95
paddleboarding       80
skiresorts           61
hot_air_balloons     61
snorkeling           38
hanggliding          17
parasailing          16
hotsprings           12
Name: category, dtype: int64

In [13]:
# determine if all datapoints for each 
yelp_df[["latitude","longitude"]].groupby(["latitude","longitude"]).size()

latitude    longitude 
-45.032192  168.661000    15
-37.814218  144.963161    19
-36.852095  174.763180    17
-36.718805  142.220951    10
-35.141813  150.391646    13
                          ..
 59.325117  18.071093     15
 59.437216  24.745369      2
 59.913330  10.738970     14
 60.167488  24.942747     15
 60.394306  5.325919      11
Length: 161, dtype: int64

In [14]:
yelp_df["latitude"].nunique()

161

In [18]:
new_df = pd.merge(yelp_df,locations_df,how="right",left_on=["latitude","longitude"],right_on=["Latitude","Longitude"])
new_df[new_df["category"].isna()]

,latitude,longitude,category,results_total,avg_rating,avg_review_count,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
2197,NaN,NaN,NaN,NaN,NaN,NaN,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871
2198,NaN,NaN,NaN,NaN,NaN,NaN,10,Dubrovnik,Croatia,Croatia,42.648736,18.094680
2199,NaN,NaN,NaN,NaN,NaN,NaN,14,Dubai,United Arab Emirates,United Arab Emirates,25.265347,55.292491
2200,NaN,NaN,NaN,NaN,NaN,NaN,16,Turks and Caicos Islands​,NaN,Turks and Caicos Islands​,21.721746,-71.552781
2201,NaN,NaN,NaN,NaN,NaN,NaN,28,Zanzibar,Tanzania,Tanzania,-6.166491,39.207431
2202,NaN,NaN,NaN,NaN,NaN,NaN,29,Mauritius​,NaN,Mauritius​,-20.275945,57.570357
2203,NaN,NaN,NaN,NaN,NaN,NaN,31,Cayman Islands​,NaN,Cayman Islands​,19.703182,-79.917463
2204,NaN,NaN,NaN,NaN,NaN,NaN,32,Seychelles​,NaN,Seychelles​,-4.657498,55.454015
2205,NaN,NaN,NaN,NaN,NaN,NaN,35,Marrakesh,Morocco,Morocco,31.625826,-7.989161
2206,NaN,NaN,NaN,NaN,NaN,NaN,140,Marrakech,NaN,Morocco,31.625826,-7.989161


In [21]:
yelp_df.to_csv("../Resources/yelpdata.csv")